## Customize Your Lineup - Trial Code
Get the python/pandas logic working for the customize page before incorporating it into the flask app/js/html.

In [55]:
import pandas as pd
import numpy as np
from scipy import stats

# #try using plotly for graphing - should help transition to later javascript graphs
# import plotly
# import plotly.plotly as py
# import plotly.figure_factory as ff
# import plotly.graph_objs as go
# #need to use Plotly API Key in order to get it to run
# plotly.tools.set_credentials_file(username='mtsuomi', api_key='KU6JDtNccJICaVmeF2gF')

from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.ext.automap import automap_base
import json

In [25]:
#create engine to the database
db = create_engine('sqlite:///../fantasy_football_2018.db')

#create inspector so can see schema of database visually
inspector = inspect(db)

#Database Schema - verify the database has the info we are expecting by printing out all the schema visually
for table_name in inspector.get_table_names():
    for column in inspector.get_columns(table_name):
        print(table_name, column.get('name'), column.get('type'))
    print()

#other options for using SQLAlchemy sort of coding instead of traditional SQL queries
# engine = create_engine('sqlite:///raw_data/hawaii.sqlite')
# Base = automap_base()
# Base.prepare(engine, reflect=True)
# session = Session(bind=engine)
# inspector = inspect(engine)

week1_ppr_actuals PLAYER TEXT
week1_ppr_actuals POS TEXT
week1_ppr_actuals TEAM TEXT
week1_ppr_actuals FPTS_PPR_ACTUAL FLOAT
week1_ppr_actuals WEEK BIGINT
week1_ppr_actuals PK INTEGER

week1_ppr_projections PLAYER TEXT
week1_ppr_projections POS TEXT
week1_ppr_projections TEAM TEXT
week1_ppr_projections FPTS_PPR_ESPN FLOAT
week1_ppr_projections FPTS_PPR_CBS FLOAT
week1_ppr_projections FPTS_PPR_SHARKS FLOAT
week1_ppr_projections FPTS_PPR_SCOUT FLOAT
week1_ppr_projections FPTS_PPR_PRVS_WK_ACTUAL FLOAT
week1_ppr_projections PK INTEGER

week1_ppr_projections_all_scraped PLAYER TEXT
week1_ppr_projections_all_scraped POS TEXT
week1_ppr_projections_all_scraped TEAM TEXT
week1_ppr_projections_all_scraped FPTS_PPR_ESPN FLOAT
week1_ppr_projections_all_scraped FPTS_PPR_CBS FLOAT
week1_ppr_projections_all_scraped FPTS_PPR_SHARKS FLOAT
week1_ppr_projections_all_scraped FPTS_PPR_SCOUT FLOAT
week1_ppr_projections_all_scraped FPTS_PPR_PRVS_WK_ACTUAL FLOAT
week1_ppr_projections_all_scraped PK INTEGER

w

##### Customize Projection Average

In [64]:
week = 3
espn_wt = 15
cbs_wt = 40
sharks_wt = 15
scout_wt = 0
prior_wt = 30
def_boost_wt = 'Full'
overunder_boost_wt = 'Partial'
twitter_boost_wt = 'None'

#convert weights to percentages
espn_pct = espn_wt/100
cbs_pct = cbs_wt/100
sharks_pct = sharks_wt/100
scout_pct = scout_wt/100
prior_pct = prior_wt/100

In [28]:
#get the projections for your week selected
stmt = f"""
        SELECT *
        FROM week{week}_ppr_projections
       """

#need to input db.session.bind in flask app due to different initalization process
df_proj = pd.read_sql_query(stmt, db)
print(df_proj.shape)
df_proj.head()

(357, 9)


,PLAYER,POS,TEAM,FPTS_PPR_ESPN,FPTS_PPR_CBS,FPTS_PPR_SHARKS,FPTS_PPR_SCOUT,FPTS_PPR_PRVS_WK_ACTUAL,PK
0,Alvin Kamara,RB,NO,21.8,21.0,20.8,30.4,17.9,1
1,Ezekiel Elliott,RB,Dal,21.1,15.0,20.6,20.7,19.7,2
2,Tom Brady,QB,NE,20.9,23.0,23.6,26.4,16.4,3
3,Todd Gurley,RB,LAR,20.9,23.0,20.9,30.5,32.3,4
4,Antonio Brown,WR,Pit,20.2,23.0,22.9,25.1,15.7,5


In [29]:
df_proj.columns

Index(['PLAYER', 'POS', 'TEAM', 'FPTS_PPR_ESPN', 'FPTS_PPR_CBS',
       'FPTS_PPR_SHARKS', 'FPTS_PPR_SCOUT', 'FPTS_PPR_PRVS_WK_ACTUAL', 'PK'],
      dtype='object')

In [31]:
#create initial custom weighted average of the five projection sources based on user input
df_proj['FPTS_PPR_CUSTOM_AVG'] = df_proj.apply(lambda row: 
                                    (row['FPTS_PPR_ESPN']*espn_pct +
                                    row['FPTS_PPR_CBS']*cbs_pct +
                                    row['FPTS_PPR_SHARKS']*sharks_pct +
                                    row['FPTS_PPR_SCOUT']*scout_pct +
                                    row['FPTS_PPR_PRVS_WK_ACTUAL']*prior_pct),
                                    axis='columns')
df_proj.head()

,PLAYER,POS,TEAM,FPTS_PPR_ESPN,FPTS_PPR_CBS,FPTS_PPR_SHARKS,FPTS_PPR_SCOUT,FPTS_PPR_PRVS_WK_ACTUAL,PK,FPTS_PPR_CUSTOM_AVG
0,Alvin Kamara,RB,NO,21.8,21.0,20.8,30.4,17.9,1,20.160
1,Ezekiel Elliott,RB,Dal,21.1,15.0,20.6,20.7,19.7,2,18.165
2,Tom Brady,QB,NE,20.9,23.0,23.6,26.4,16.4,3,20.795
3,Todd Gurley,RB,LAR,20.9,23.0,20.9,30.5,32.3,4,25.160
4,Antonio Brown,WR,Pit,20.2,23.0,22.9,25.1,15.7,5,20.375


##### Boost Factor Logic - Import Data (later DB connection), Merge Data, then Do Logic

In [33]:
### FIGURE OUT UPDATED DB CONNECTION LATER ###
#import the data for now
df_opp_defense = pd.read_excel('../def_scoring_proj.xlsx')
df_overunder = pd.read_csv('../week_2_over_unders.csv')
df_tweets = pd.read_csv('../expert_tweets_week_3.csv')

In [34]:
df_opp_defense.head(3)

,"PLAYER, TEAM POS",team_abbr,week_2_opp,week_2_proj,week_2_actual,week_3_opp,week_3_proj
0,Jets D/ST D/ST,NYJ,Mia,4.9,10,Cle,5.2
1,Vikings D/ST D/ST,Min,GB,6.4,2,Buf,13.2
2,Ravens D/ST D/ST,Bal,Cin,6.1,-2,Den,6.7


In [35]:
df_overunder.head(3)

,team,over_under,team_abbr
0,Cleveland Browns,49.0,Cle
1,New Orleans Saints,49.0,NO
2,Indianapolis Colts,48.5,Ind


In [36]:
df_tweets.head(3)

,Unnamed: 0,number,player,compound_score,binned,expert_sentiment,color
0,0,1,Cam Newton,-0.223550,"(-0.5, -0.02]",Concerned,yellow
1,1,2,Drew Brees,0.168067,"(0.02, 0.5]",Good,blue
2,2,3,Aaron Rodgers,0.062883,"(0.02, 0.5]",Good,blue


In [39]:
#Connor's code for binning the over-unders
high = df_overunder['over_under'].max()
print(high)
low = df_overunder['over_under'].min()
print(low)
mean = df_overunder['over_under'].mean()
print(mean)

#create bins from range of over unders
bins = [(low-.5), ((low+low+mean)/3), ((low+mean+mean)/3), ((high+mean+mean)/3), ((high+high+mean)/3), (high+.5)]
df_overunder['binned'] = pd.cut(df_overunder['over_under'], bins)
df_overunder.head()

#convert binned values to string so we can categorize with adjectives
df_overunder['binned'] = df_overunder['binned'].map(lambda x: str(x))

#create adjectives from bins
score_bins = {
    '(42.0, 43.521]': 'Terrible',
    '(43.521, 44.542]': 'Concerned',
    '(44.542, 48.208]': 'Neutral',
    '(48.208, 50.854]': 'Good',
    '(50.854, 54.0]': 'Awesome'
}

df_overunder['score_sentiment'] = df_overunder['binned'].map(score_bins)

color_bins = {
    'Terrible': 'red',
    'Concerned': 'yellow',
    'Neutral': 'grey',
    'Good': 'blue',
    'Awesome': 'green'
}

df_overunder['color'] = df_overunder['score_sentiment'].map(color_bins)
df_overunder

53.5
42.5
45.5625


,team,over_under,team_abbr,binned,score_sentiment,color
0,Cleveland Browns,49.0,Cle,"(48.208, 50.854]",Good,blue
1,New Orleans Saints,49.0,NO,"(48.208, 50.854]",Good,blue
2,Indianapolis Colts,48.5,Ind,"(48.208, 50.854]",Good,blue
3,Washington Redskins,48.5,Wsh,"(48.208, 50.854]",Good,blue
4,Minnesota Vikings,46.5,Min,"(44.542, 48.208]",Neutral,grey
5,Green Bay Packers,46.5,GB,"(44.542, 48.208]",Neutral,grey
6,Los Angeles Chargers,43.0,LAC,"(42.0, 43.521]",Terrible,red
7,Buffalo Bills,43.0,Buf,"(42.0, 43.521]",Terrible,red
8,Kansas City Chiefs,53.5,KC,"(50.854, 54.0]",Awesome,green
9,Pittsburgh Steelers,53.5,Pit,"(50.854, 54.0]",Awesome,green


In [57]:
df_opp_defense.head(3)

,"PLAYER, TEAM POS",team_abbr,week_2_opp,week_2_proj,week_2_actual,week_3_opp,week_3_proj
0,Jets D/ST D/ST,NYJ,Mia,4.9,10,Cle,5.2
1,Vikings D/ST D/ST,Min,GB,6.4,2,Buf,13.2
2,Ravens D/ST D/ST,Bal,Cin,6.1,-2,Den,6.7


In [ ]:
PLAYER, TEAM POS	team_abbr	week_2_opp	week_2_proj	week_2_actual	week_3_opp	week_3_proj

In [61]:
#merge on the boost factor data to players
#data: opposing defense, opposing defense bin, twitter sentiment, twitter sentiment bin, vegas overunder, vegas overunder bin

#check size of starting df
print(df_proj.shape)

#first merge in the twitter sentiment - always do left merge (rather have Nans for boost factors than lose players)
df_boosts = pd.merge(df_proj, df_tweets[['player', 'compound_score', 'expert_sentiment']],
                     how='left', left_on='PLAYER', right_on='player')
df_boosts.drop('player', axis='columns', inplace=True) #don't need to keep this added player name column
print(df_boosts.shape)

#second merge in the overunder - always do left merge (rather have Nans for boost factors than lose players)
df_boosts = pd.merge(df_boosts, df_overunder[['team_abbr', 'over_under', 'score_sentiment']],
                     how='left', left_on='TEAM', right_on='team_abbr')
df_boosts.drop('team_abbr', axis='columns', inplace=True) #don't need to keep this added team abbr column
print(df_boosts.shape)

#third merge in the defense projections - always do left merge (rather have Nans for boost factors than lose players)
df_boosts = pd.merge(df_boosts, df_opp_defense[['team_abbr', 'week_3_opp', 'week_3_proj']],
                     how='left', left_on='TEAM', right_on='team_abbr')
df_boosts.drop('team_abbr', axis='columns', inplace=True) #don't need to keep this added team abbr column
print(df_boosts.shape)

#convert any of the Nans in our binned sentiment columns to Neutral (that will just apply a zero boost to those players)
df_boosts.fillna('Neutral', inplace=True)

df_boosts.head(30)

(357, 10)
(357, 12)
(357, 14)
(357, 16)


,PLAYER,POS,TEAM,FPTS_PPR_ESPN,FPTS_PPR_CBS,FPTS_PPR_SHARKS,FPTS_PPR_SCOUT,FPTS_PPR_PRVS_WK_ACTUAL,PK,FPTS_PPR_CUSTOM_AVG,compound_score,expert_sentiment,over_under,score_sentiment,week_3_opp,week_3_proj
0,Alvin Kamara,RB,NO,21.8,21.0,20.8,30.4,17.9,1,20.160,-0.09417,Concerned,49,Good,Atl,4.3
1,Ezekiel Elliott,RB,Dal,21.1,15.0,20.6,20.7,19.7,2,18.165,0.296,Good,42.5,Terrible,Sea,6.5
2,Tom Brady,QB,NE,20.9,23.0,23.6,26.4,16.4,3,20.795,0.00398333,Neutral,45.5,Neutral,Det,5.1
3,Todd Gurley,RB,LAR,20.9,23.0,20.9,30.5,32.3,4,25.160,0.123336,Good,44.5,Concerned,LAC,6.8
4,Antonio Brown,WR,Pit,20.2,23.0,22.9,25.1,15.7,5,20.375,-0.046625,Concerned,53.5,Awesome,TB,5.5
5,Ryan Fitzpatrick,QB,TB,19.7,20.0,20.5,25.0,30.0,6,23.030,0.0424545,Good,44,Concerned,Pit,3.6
6,Patrick Mahomes,QB,KC,19.5,23.0,22.4,28.0,38.8,7,27.125,0.0619618,Good,53.5,Awesome,SF,6.2
7,David Johnson,RB,Ari,19.4,15.0,16.0,17.9,6.1,8,13.140,0.0807625,Good,44.5,Concerned,Chi,7
8,Michael Thomas,WR,NO,19.4,21.0,21.8,22.3,30.9,9,23.850,0.0767,Good,49,Good,Atl,4.3
9,Aaron Rodgers,QB,GB,19.3,24.0,21.2,25.3,16.0,10,20.475,0.062883,Good,46.5,Neutral,Wsh,4.5


In [62]:
#apply defensive boost factor
# def_boost = 'Full'
# overunder_boost = 'Partial'
# twitter_boost = 'None'

player = 'Alvin Kamara'
team = 'NO'
custom_points = 20.16
expert_sentiment = 'Concerned'
score_sentiment = 'Good'
defense_sentiment = 'Terrible'

In [63]:
boost_multipliers = {'opp_defense': {'Awesome': -0.8, 'Good': -0.4, 'Neutral': 0, 'Concerned': 0.4, 'Terrible': 0.8},
                     
                     
                    }

In [ ]:
#will need to iterate over this for each boost condition
boost_points = custom_points * 0.2 (boost multiplier condition)  * confidence_level 
custom_points_boost = custom_points + boost_points

In [81]:
def over_under_boost_calculator(row, user_boost_weight):
    #define the user's boost multiplier based on their selection of how much of the factor to apply
    if user_boost_weight == 'Full':
        user_boost_multiplier = 1.0
    elif user_boost_weight == 'Partial':
        user_boost_multiplier = 0.5
    else:
        user_boost_multiplier = 0
        
    #these are our custom boost factors based on our o/u score sentiments (can change these on the backend however we want)
    over_under_boost_values = {'Awesome': 0.2, 'Good': 0.1, 'Neutral': 0, 'Concerned': -0.1, 'Terrible': -0.2}
    
    #get the starting custom weighted average fantasy points projections by the user
    custom_points = row['FPTS_PPR_CUSTOM_AVG']
    
    #get boost multiplier based on the overunder binned sentiments and our dictionary of boost factors
    boost_multiplier = over_under_boost_values.get(row['score_sentiment'], 0)
    
    #calculate the boost points
    over_under_boost_points = custom_points * boost_multiplier * user_boost_multiplier
    
    return over_under_boost_points

In [89]:
def twitter_boost_calculator(row, user_boost_weight):
    #define the user's boost multiplier based on their selection of how much of the factor to apply
    if user_boost_weight == 'Full':
        user_boost_multiplier = 1.0
    elif user_boost_weight == 'Partial':
        user_boost_multiplier = 0.5
    else:
        user_boost_multiplier = 0
        
    #these are our custom boost factors based on our twitter sentiments (can change these on the backend however we want)
    twitter_boost_values = {'Awesome': 0.2, 'Good': 0.1, 'Neutral': 0, 'Concerned': -0.1, 'Terrible': -0.2}
    
    #get the starting custom weighted average fantasy points projections by the user
    custom_points = row['FPTS_PPR_CUSTOM_AVG']
    
    #get boost multiplier based on the overunder binned sentiments and our dictionary of boost factors
    boost_multiplier = twitter_boost_values.get(row['expert_sentiment'], 0)
    
    #calculate the boost points
    twitter_boost_points = custom_points * boost_multiplier * user_boost_multiplier
    
    return twitter_boost_points

In [96]:
def defense_boost_calculator(row, user_boost_weight):
    #if the player looking it as a Defense, we will want to skip and not boost the defense projection
    if row['POS'] == 'D/ST':
        return 0.0
    
    #otherwise apply the defense boost to all other positions (may in future want to consider removing kickers as
    #they are often unpredictable, and maybe the correlation of a harder defense is even positive on kicker points b/c they kick more FGs than TDs)
    else:
        #define the user's boost multiplier based on their selection of how much of the factor to apply
        if user_boost_weight == 'Full':
            user_boost_multiplier = 1.0
        elif user_boost_weight == 'Partial':
            user_boost_multiplier = 0.5
        else:
            user_boost_multiplier = 0

        #these are our custom boost factors based on our opposing defenses(can change these on the backend however we want)
        #if opposing defense is awesome, that has a negative impact on our fantasy players
        opp_defense_boost_values = {'Awesome': -0.2, 'Good': -0.1, 'Neutral': 0, 'Concerned': 0.1, 'Terrible': 0.2}

        #get the starting custom weighted average fantasy points projections by the user
        custom_points = row['FPTS_PPR_CUSTOM_AVG']

        #get boost multiplier based on the overunder binned sentiments and our dictionary of boost factors
        boost_multiplier = opp_defense_boost_values.get(row['opp_defense_sentiment'], 0)

        #calculate the boost points
        defense_boost_points = custom_points * boost_multiplier * user_boost_multiplier

        return defense_boost_points

In [71]:
df_boosts['score_sentiment'][0]

'Good'

In [77]:
over_under_boost_values = {'Awesome': 0.2, 'Good': 0.1, 'Neutral': 0, 'Concerned': -0.1, 'Terrible': -0.2}

over_under_boost_values.get("", 0.0)


0.0

In [70]:
df_boosts['FPTS_PPR_CUSTOM_AVG'][0]

20.159999999999997

In [68]:
overunder_boost_wt

'Partial'

In [79]:
df_boosts['OVER_UNDER_BOOST'] = df_boosts.apply(lambda row: 
                                    over_under_boost_calculator(row, overunder_boost_wt),
                                    axis='columns')

In [90]:
df_boosts['TWITTER_BOOST'] = df_boosts.apply(lambda row: 
                                    twitter_boost_calculator(row, twitter_boost_wt),
                                    axis='columns')

In [97]:
df_boosts['OPPOSING_DEFENSE_BOOST'] = df_boosts.apply(lambda row: 
                                        defense_boost_calculator(row, def_boost_wt),
                                        axis='columns')

KeyError: ('opp_defense_sentiment', 'occurred at index 0')

In [91]:
df_boosts.head()

,PLAYER,POS,TEAM,FPTS_PPR_ESPN,FPTS_PPR_CBS,FPTS_PPR_SHARKS,FPTS_PPR_SCOUT,FPTS_PPR_PRVS_WK_ACTUAL,PK,FPTS_PPR_CUSTOM_AVG,compound_score,expert_sentiment,over_under,score_sentiment,week_3_opp,week_3_proj,OVER_UNDER_BOOST,TWITTER_BOOST
0,Alvin Kamara,RB,NO,21.8,21.0,20.8,30.4,17.9,1,20.160,-0.09417,Concerned,49,Good,Atl,4.3,1.0080,-2.0160
1,Ezekiel Elliott,RB,Dal,21.1,15.0,20.6,20.7,19.7,2,18.165,0.296,Good,42.5,Terrible,Sea,6.5,-1.8165,1.8165
2,Tom Brady,QB,NE,20.9,23.0,23.6,26.4,16.4,3,20.795,0.00398333,Neutral,45.5,Neutral,Det,5.1,0.0000,0.0000
3,Todd Gurley,RB,LAR,20.9,23.0,20.9,30.5,32.3,4,25.160,0.123336,Good,44.5,Concerned,LAC,6.8,-1.2580,2.5160
4,Antonio Brown,WR,Pit,20.2,23.0,22.9,25.1,15.7,5,20.375,-0.046625,Concerned,53.5,Awesome,TB,5.5,2.0375,-2.0375


In [ ]:
def_boost_wt = 'Full'
overunder_boost_wt = 'Partial'
twitter_boost_wt = 'None'


##### Converting Our DF to a JSON File
Note: when you return the json in flask, you'll also want to jsonify it to make sure it is formatted well.  return jsonify(json)

In [12]:
#create json that is a list of dictionaries, where each row is its own dictionary with column:value pairs
df_week1_ppr_projections.head(10).to_json('filename_test', orient='records')


In [56]:
json_test = df_week1_ppr_projections.head(10).to_json(orient='records')
jsonfiles = json.loads(json_test)

jsonfiles

[{'PLAYER': 'Alvin Kamara',
  'POS': 'RB',
  'TEAM': 'NO',
  'FPTS_PPR_ESPN': 22.2,
  'FPTS_PPR_CBS': 19.0,
  'FPTS_PPR_SHARKS': 19.1,
  'FPTS_PPR_SCOUT': 34.0,
  'FPTS_PPR_PRVS_WK_ACTUAL': 0.0,
  'PK': 1},
 {'PLAYER': 'David Johnson',
  'POS': 'RB',
  'TEAM': 'Ari',
  'FPTS_PPR_ESPN': 21.3,
  'FPTS_PPR_CBS': 19.0,
  'FPTS_PPR_SHARKS': 23.6,
  'FPTS_PPR_SCOUT': 27.0,
  'FPTS_PPR_PRVS_WK_ACTUAL': 0.0,
  'PK': 2},
 {'PLAYER': 'Todd Gurley',
  'POS': 'RB',
  'TEAM': 'LAR',
  'FPTS_PPR_ESPN': 21.2,
  'FPTS_PPR_CBS': 23.0,
  'FPTS_PPR_SHARKS': 19.3,
  'FPTS_PPR_SCOUT': 28.5,
  'FPTS_PPR_PRVS_WK_ACTUAL': 0.0,
  'PK': 3},
 {'PLAYER': 'Antonio Brown',
  'POS': 'WR',
  'TEAM': 'Pit',
  'FPTS_PPR_ESPN': 19.5,
  'FPTS_PPR_CBS': 22.0,
  'FPTS_PPR_SHARKS': 19.3,
  'FPTS_PPR_SCOUT': 17.3,
  'FPTS_PPR_PRVS_WK_ACTUAL': 0.0,
  'PK': 4},
 {'PLAYER': 'Tom Brady',
  'POS': 'QB',
  'TEAM': 'NE',
  'FPTS_PPR_ESPN': 19.4,
  'FPTS_PPR_CBS': 20.0,
  'FPTS_PPR_SHARKS': 27.4,
  'FPTS_PPR_SCOUT': 23.3,
  'FPTS_PP

In [51]:
jsonify(json)

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.